# Logistic regression

## Visualizing the data

In [ ]:
# 在开始实现任何学习算法之前，如果可能的话，最好将数据可视化
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
names = ['exam1', 'exam2', 'admitted']
data = pd.read_csv('ex2data1.txt', names=names)
data

In [ ]:
data.describe()

In [ ]:
# 两个分数的散点图，并使用颜色编码来可视化，如果样本是正的（被接纳）或负的（未被接纳）
positive = data[data['admitted'] == 1]
negetive = data[data['admitted'] == 0]

fig, ax = plt.subplots(figsize=(6, 5))
ax.scatter(positive['exam1'], positive['exam2'],
           color='b', marker='o', label='Admitted')
ax.scatter(negetive['exam1'], negetive['exam2'],
           color='r', marker='x', label='Not Admitted')
# 设置图例显示在图的上方
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width, box.height * 0.8])
ax.legend(loc='center left', bbox_to_anchor=(0.2, 1.12), ncol=3)
# 设置横纵坐标名
ax.set_xlabel('Exam 1 Score')
ax.set_ylabel('Exam 2 Score')
plt.show()
# 看起来在两类间，有一个清晰的决策边界。现在我们需要实现逻辑回归，那样就可以训练一个模型来预测结果

## Sigmoid function

In [ ]:
def sigmoid(z):
    return 1/(1+np.exp(-z))


# 做一个快速的检查，来确保它可以工作
x1 = np.arange(-10, 10, 0.1)
plt.plot(x1, sigmoid(x1),'r')
plt.show()

## Cost function

In [ ]:
# 逻辑回归的代价函数
def cost(theta, X, y):
    return ((-y)*np.log(sigmoid(X.dot(theta.T)))-(1-y)*np.log(1-sigmoid(X.dot(theta.T)))).mean()

In [ ]:
# add a ones column - this makes the matrix multiplication work out easier
if 'Ones' not in data.columns:
    data.insert(0, 'Ones', 1)

# set X (training data) and y (target variable)
# Convert the frame to its Numpy-array representation.
X = np.array(data.iloc[:, :-1])
# Return is NOT a Numpy-matrix, rather, a Numpy-array.
y = np.array(data.iloc[:, -1])

theta = np.zeros(X.shape[1])

In [ ]:
cost(theta,X,y)

## Gradient

In [ ]:
def gradient(theta, X, y):
    return X.T.dot((sigmoid(X.dot(theta.T))-y))/len(y)


# the gradient of the cost is a vector of the same length as θ where the jth element (for j = 0, 1, . . . , n)
gradient(theta, X, y)

## Learning θ parameters

In [ ]:
import scipy.optimize as opt
result = opt.fmin_tnc(func=cost, x0=theta, fprime=gradient, args=(X, y))
result

## Evaluating logistic regression

In [ ]:
def predict(theta, X):
    probability = sigmoid(X.dot(theta.T))
    return [1 if i >= 0.5 else 0 for i in probability]  # return a list

In [ ]:
final_theta = result[0]
predictions = predict(final_theta, X)
correct = [1 if a == b else 0 for (a, b) in zip(predictions, y)]
accuracy = sum(correct)/len(correct)
accuracy

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(predictions, y))

## Decision boundary（决策边界）

In [ ]:
# Decision boundary
x1 = np.arange(130, step=0.1)
x2 = -(final_theta[0]+final_theta[1]*x1)/final_theta[2]

fig, ax = plt.subplots(figsize=(8, 5))
ax.scatter(positive['exam1'], positive['exam2'],
           color='b', marker='o', label='Admitted')
ax.scatter(negetive['exam1'], negetive['exam2'],
           color='r', marker='x', label='Not Admitted')
ax.plot(x1, x2,color='orange')
ax.set_xlim(0, 130)
ax.set_ylim(0, 130)
ax.set_xlabel('x1')
ax.set_ylabel('x2')
ax.set_title('Decision Boundary')
plt.show()

# Regularized logistic regression

## Visualizing the data

In [ ]:
names = ['Test 1', 'Test 2', 'Accepted']
data2 = pd.read_csv('ex2data2.txt', names=names)
data2.head()

In [ ]:
def plot_data2():
    positive = data2[data2['Accepted'].isin([1])]
    negative = data2[data2['Accepted'].isin([0])]

    fig, ax = plt.subplots(figsize=(8, 5))
    ax.scatter(positive['Test 1'], positive['Test 2'],
               s=50, c='b', marker='o', label='Accepted')
    ax.scatter(negative['Test 1'], negative['Test 2'],
               s=50, c='r', marker='x', label='Rejected')
    ax.legend()
    ax.set_xlabel('Test 1 Score')
    ax.set_ylabel('Test 2 Score')


plot_data2()
# 注意到其中的正负两类数据并没有线性的决策界限。因此直接用logistic回归在这个数据集上并不能表现良好，因为它只能用来寻找一个线性的决策边界

## Feature mapping
一个拟合数据的更好的方法是从每个数据点创建更多的特征。
我们将把这些特征映射到所有的x1和x2的多项式项上，直到第六次幂。

In [ ]:
def feature_mapping(x1, x2, power):
    data = {}
    for i in np.arange(power+1):
        for p in np.arange(i+1):
            data["f{}{}".format(i-p, p)] = np.power(x1, i-p)*np.power(x2, p)
#     data = {"f{}{}".format(i - p, p): np.power(x1, i - p) * np.power(x2, p)
#                 for i in np.arange(power + 1)
#                 for p in np.arange(i + 1)
#             }
    return pd.DataFrame(data)

In [ ]:
x1 = np.array(data2['Test 1'])
x2 = np.array(data2['Test 2'])
_data2 = feature_mapping(x1, x2, power=6)
_data2.head()

经过映射，我们将有两个特征的向量转化成了一个28维的向量。

在这个高维特征向量上训练的logistic回归分类器将会有一个更复杂的决策边界，当我们在二维图中绘制时，会出现非线性。

虽然特征映射允许我们构建一个更有表现力的分类器，但它也更容易过拟合。在接下来的练习中，我们将实现正则化的logistic回归来拟合数据，并且可以看到正则化如何帮助解决过拟合的问题。

## Regularized Cost function

In [ ]:
# 这里因为做特征映射的时候已经添加了偏置项，所以不用手动添加了。
X = np.array(_data2)
y = np.array(data2['Accepted'])
theta = np.zeros(X.shape[1])

In [ ]:
def costReg(theta, X, y, l=1):
    # 不惩罚第一项
    _theta = theta[1:]
    reg = (l/(2*len(y)))*(_theta.dot(_theta.T))
    return cost(theta, X, y)+reg

In [ ]:
costReg(theta, X, y, l=1) 

## Regularized gradient

In [ ]:
def gradientReg(theta,X,y,l=1):
    reg=(l/len(y))*theta
    reg[0]=0
    return gradient(theta,X,y)+reg

In [ ]:
gradientReg(theta, X, y, 1)

## Learning parameters

In [ ]:
result2 = opt.fmin_tnc(func=costReg, x0=theta, fprime=gradientReg, args=(X, y, 2))
result2

In [ ]:
# 使用高级Python库scikit-learn来解决这个问题
from sklearn import linear_model # 调用sklearn的线性回归包
model = linear_model.LogisticRegression(penalty='l2', C=1.0)
model.fit(X, y.ravel())

linear_model.LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

model.score(X, y) 

## Evaluating logistic regression

In [ ]:
final_theta = result2[0]
predictions = predict(final_theta, X)
correct = [1 if a==b else 0 for (a, b) in zip(predictions, y)]
accuracy = sum(correct) / len(correct)
accuracy

In [ ]:
print(classification_report(y, predictions))

## Decision boundary（决策边界）

In [ ]:
x=np.linspace(-1,1.5,250)
xx,yy=np.meshgrid(x,x)

z=np.array(feature_mapping(xx.ravel(),yy.ravel(),6))
z=z.dot(final_theta)
z=z.reshape(xx.shape)

plot_data2()
plt.contour(xx,yy,z,0)
plt.ylim(-0.8,1.2)